# inference on Docker Image


In [8]:
import boto3
import sagemaker
import pandas as pd
import os

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [9]:
%store -r 

In [10]:
# training_job_name = 'bert2tweet-2020-07-08-07-58-27-895'
print(training_job_name)

tensorflow-training-2020-07-23-13-02-11-499


In [11]:
# inference_image = '057716757052.dkr.ecr.us-west-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu'
inference_image = ecr_infer_custom_image_tf_serving_20_cpu
print("inference_image: ", inference_image)

inference_image:  057716757052.dkr.ecr.us-west-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu


In [12]:
from sagemaker.tensorflow.serving import Model

model = Model(model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
              role=role,
              entry_point='inference.py',
              image = inference_image
             ) 

In [13]:
%%time

instance_type='ml.m4.xlarge'
deployed_model = model.deploy(initial_instance_count = 1,
                             instance_type = instance_type,
                             wait=True)


-------------!CPU times: user 1min 2s, sys: 10.4 s, total: 1min 12s
Wall time: 7min 45s


## Invoke the endpoint without custom Docker image

In [14]:
# tweet_bert_endpoint_name = 'sagemaker-tensorflow-serving-2020-07-09-00-37-50-247'
tweet_bert_endpoint_name = deployed_model.endpoint
print(tweet_bert_endpoint_name)

sagemaker-tensorflow-serving-2020-07-24-10-34-12-001


In [30]:
import json
from sagemaker.tensorflow.serving import Predictor

predictor = Predictor(endpoint_name = tweet_bert_endpoint_name,
                      sagemaker_session = sess,
                      content_type = 'application/json',
                     )

## Inference 실행

In [35]:
from TweetUtil import TweetUtil

tweet_util = TweetUtil()
tweet_util.load_emoji_data('emoji_to_idx.pickle')
emoji = tweet_util.get_emo_class_label(3)
print(emoji)

emoji_to_idx is loaded
😂


In [36]:
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
sample_df = test_df.sample(10)
sample_df

,TWEET,LABEL
12305,@freddyamazin alicia keys without makeup leave...,5
11966,@gentsrules i am literally weak,3
15424,@harrisjofficial come to nyc soon babes,6
22199,you're pathetic you like get a grip,8
21399,i did hear an interesting point though which w...,9
23503,@fantagiosmusic hellovenus wiggle wiggle lyric...,8
11933,4 am and just now laying down for bed,6
7052,"back of the class there is always such dude,.....",4
601,@uswntaddict17 allie_long dang girl....kill it!!!,2
9345,power gets me too amped y'all don't even knowwwww,6


In [37]:
import tensorflow as tf
import json

columns = ['TWEET', 'LABEL']
for tweet, label in zip(sample_df.TWEET.values, sample_df.LABEL.values):
    # print("label: {}, tweet: {}".format(label, tweet))
    
    reviews = [tweet]

    predicted_classes = predictor.predict(reviews)[0]
    print('Prediction, Ground_truth- {}:{}, {}:{} \n tweet: {}'.format(
        predicted_classes, 
        tweet_util.get_emo_class_label(predicted_classes),
        label, 
        tweet_util.get_emo_class_label(label),        
        tweet))    

        



Prediction, Ground_truth- [0.134469762, -0.121591263, -0.0284810774, 0.334820896, 0.178134874, -0.0273167379, -0.0720925778, 0.0546125583, -0.0864444, 0.130811]:None, 5:😍 
 tweet: @freddyamazin alicia keys without makeup leaves me speechless
Prediction, Ground_truth- [0.129771516, -0.131948397, -0.0391965, 0.356477648, 0.179618597, -0.0302035399, -0.0877474621, 0.0599169359, -0.0807562768, 0.135087952]:None, 3:😂 
 tweet: @gentsrules i am literally weak
Prediction, Ground_truth- [0.123269968, -0.144477308, -0.0325035639, 0.367093325, 0.171925306, -0.0502512641, -0.0921853334, 0.0646842271, -0.0628928766, 0.144412324]:None, 6:😩 
 tweet: @harrisjofficial come to nyc soon babes
Prediction, Ground_truth- [0.101533674, -0.143657267, -0.0054078144, 0.355702549, 0.15496093, -0.0614906102, -0.0590389706, 0.0832456425, -0.0256584939, 0.164407343]:None, 8:🙄 
 tweet: you're pathetic you like get a grip
Prediction, Ground_truth- [0.0667983443, -0.143886477, 0.00691256113, 0.308830231, 0.11478053, -

## Return Top 1 on another serializer

In [ ]:
def show_top_N_label(score_list, topN):

    import numpy as np

    top_n_idx = np.argsort(score_list)[-topN:]
    top_n_values = [score_list[i] for i in top_n_idx]
    
    print(top_n_idx)


    top_n_idx_list = top_n_idx.tolist()
    top_n_idx_list.reverse()
    top_n_values = [score_list[i] for i in top_n_idx_list]    
    
    print(top_n_idx_list)
    print(top_n_values)

topN = 3    
show_top_N_label(predicted_classes[0], topN)


## Boto3 Client 에서 실행

In [25]:
%%bash -s {invoke_endpoint}

invoke_endpoint=$1
echo $invoke_endpoint

aws sagemaker-runtime invoke-endpoint \
  --endpoint-name  $invoke_endpoint \
  --body '["This is great"]' \
  --content-type application/json \
  --accept application/json \
  results
echo $results

sagemaker-tensorflow-serving-2020-07-24-10-34-12-001
{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}



In [ ]:
payload = "This is great"
payload = '["' + payload + '"]'
print(payload)

In [ ]:
import json
import os
import io
import boto3
import csv

ENDPOINT_NAME = os.environ['ENDPOINT_NAME'] 
runtime = boto3.client('runtime.sagemaker')


def lambda_handler(event, context):
    # TODO implement
try:
print("Received event: " + json.dumps(event, indent=2))

data = json.loads(json.dumps(event))
payload = data['tweet']
payload = '["' + payload + '"]'
print("Final input: " ,payload)

print(payload)

response = runtime.invoke_endpoint(EndpointName = ENDPOINT_NAME,
                            ContentType = 'application/json',
                            Body = payload)

print(response)
result = json.loads(response['Body'].read().decode())
print(result)
return result
except:
print("An error occurred")
return data


# return {
#     'statusCode': 200,
#     'body': json.dumps('Hello from Lambda!')
# }
